In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [3]:
from bs4 import BeautifulSoup

CHANGING THE TABLE ON THE WEBSITE INTO JSON FORMAT

In [4]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
print(df[0].to_json(orient='records'))

[{"Postcode":"M1A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postcode":"M2A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postcode":"M3A","Borough":"North York","Neighbourhood":"Parkwoods"},{"Postcode":"M4A","Borough":"North York","Neighbourhood":"Victoria Village"},{"Postcode":"M5A","Borough":"Downtown Toronto","Neighbourhood":"Harbourfront"},{"Postcode":"M5A","Borough":"Downtown Toronto","Neighbourhood":"Regent Park"},{"Postcode":"M6A","Borough":"North York","Neighbourhood":"Lawrence Heights"},{"Postcode":"M6A","Borough":"North York","Neighbourhood":"Lawrence Manor"},{"Postcode":"M7A","Borough":"Queen's Park","Neighbourhood":"Not assigned"},{"Postcode":"M8A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postcode":"M9A","Borough":"Etobicoke","Neighbourhood":"Islington Avenue"},{"Postcode":"M1B","Borough":"Scarborough","Neighbourhood":"Rouge"},{"Postcode":"M1B","Borough":"Scarborough","Neighbourhood":"Malvern"},{"Postcode":"M2B","Borough":"No

In [6]:
#define columns for the dataframe
column_names = ['PostalCode', 'Borough', 'Neighborhood']
neighborhood = pd.DataFrame(columns = column_names )

#transforming data into pandas dataframe
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
PostalCode = df["Postcode"].tolist()
Borough = df["Borough"].tolist()
Neighborhood = df["Neighbourhood"].tolist()

# removing cells with 'Not assigned' value in Borough column
df = df[df.Borough != 'Not assigned']
df = df.reset_index (drop = True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [7]:
#Combining neighborhoods with the same postal code
df1 = df.groupby(['Postcode', 'Borough'], sort = False) .agg(','.join)
df1.reset_index(inplace = True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [8]:
#checking 'Not assigned' values in the Neighbourhood column
df1[df1['Neighbourhood'].isin(['Not assigned'])]

,Postcode,Borough,Neighbourhood
4,M7A,Queen's Park,Not assigned


In [9]:
#Replacing 'Not assigned' value to Name of the Borough 'Queen's Park
df1 = df1.replace('Not assigned', "Queen's Park")
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [10]:
df1.shape

(103, 3)


READING CSV FILE FOR GEOGRAPHICAL COORDINATES FOR EACH OF THE POSTAL CODES

In [11]:
Coordinate_df = pd.read_csv('http://cocl.us/Geospatial_data')

Coordinate_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
Coordinate_df.rename(columns ={'Postal Code': 'Postcode'}, inplace = True)
Coordinate_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


MERGING THE COORDINATE DATAFRAME WITH DATAFRAME CONTAINING BOROUGHS AND NEIGHBOURHOODS

In [22]:
Coordinate_df1 = pd.merge(df1, Coordinate_df, on = 'Postcode')
Coordinate_df1.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


TOTAL NUMBER OF BOROUGHS AND NEIGHBORHOODS IN OUR DATAFRAME

In [23]:
print('The data frame has {} Boroughs and {} Neighborhoods'.format (len(Coordinate_df1['Borough'].unique()), Coordinate_df1.shape[0]))

The data frame has 11 Boroughs and 103 Neighborhoods


DETRMINING THE COORDINATES OF TORONTO

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


WORKING WITH BOROUGHS CONTAINING THE WORD 'TORONTO'

In [ ]:
Toronto=df8[df8['Borough'].str.contains('Toronto')]

In [25]:
Toronto_df = Coordinate_df1[Coordinate_df1['Borough'].str.contains ('Toronto')]


In [26]:
Toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [28]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Coordinate_df1['Latitude'], Coordinate_df1['Longitude'], Coordinate_df1['Borough'], Coordinate_df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto